<h2>Important Packages</h2>

In [21]:
import pandas as pd
import numpy as np

<h2>Load data and make train and test set</h2>

In [22]:
import os
from typing import List

def get_file_list(data_folder_path: str) -> List[str]:
    file_lst = []
    
    # Get all files in the directory
    for root, _, files in os.walk(data_folder_path):
        for file in files:
            if file.endswith('.csv',):  # Only consider CSV files
                file_path = os.path.join(root, file)
                # Append file path and its modification time
                file_lst.append(file_path)


    # Extract just the file paths
    return file_lst

In [56]:
data_folder = "data"
data_files = get_file_list('data')
df_list = [pd.read_csv(data_file) for data_file in data_files]

In [61]:
df_list[4]

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,Del
0,27.5,27.2,28.7,29.0,29.0,29.0,29.0,27.2,28.4,32.2,27.2,27.2,27.4,39.7,26.8,0
1,27.5,27.2,28.7,28.9,29.2,29.0,29.1,27.2,28.3,32.5,27.2,27.2,27.4,39.6,26.7,-3
2,27.3,27.2,28.7,28.8,29.0,29.0,29.1,27.2,28.3,32.4,27.1,27.2,27.6,39.6,26.8,-3
3,27.5,27.1,28.7,28.6,29.1,29.1,29.1,27.2,28.3,32.5,27.1,27.2,27.6,39.6,26.6,-3
4,27.5,27.1,28.5,28.7,29.1,28.9,29.1,27.2,28.3,32.5,27.1,27.2,27.6,39.6,26.8,-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,29.9,32.8,37.1,37.9,38.9,38.9,38.9,32.4,37.3,62.5,31.1,32.9,31.4,55.5,30.6,42
346,29.9,32.7,37.3,38.0,38.9,38.9,38.9,32.4,37.2,62.5,31.1,32.9,31.4,55.6,30.4,42
347,30.1,33.0,37.3,38.0,38.9,38.9,38.9,32.3,37.1,62.3,31.1,32.9,31.4,55.5,30.4,43
348,29.8,32.9,37.3,38.0,38.9,38.9,38.9,32.3,37.1,62.4,31.1,32.8,31.4,55.2,30.4,42


<h4>Prepare data for LSTM model</h4>

In [62]:
from sklearn.model_selection import train_test_split

def save_past_dependence_merged_data(df_lst, window_size: int = 10):
    """
    Prepares and saves data for LSTM model with a sliding window approach.

    Parameters:
    - window_size: int, the number of past observations to consider for each sequence.
    """
    X_data, y_data = [], []

    save_path_train = 'temp/train_compressed.npz'
    save_path_test = 'temp/test_compressed.npz'

    for df in df_lst:
        df = df.dropna().reset_index(drop=True)  # Drop NaN values and reset index if needed
        m, n = df.shape
        
        # Generate sequences for the current dataframe
        for i in range(m - window_size):
            # Extract window of features (excluding the first column as features)
            X = df.iloc[i:i + window_size, :-1].values  # Use all columns except last one
            y = df.iloc[i + window_size, -1]  # Use the target column as last column

            X_data.append(X)
            y_data.append(y)

    # Convert lists to arrays
    X_data = np.array(X_data)
    y_data = np.array(y_data)

    trainX, testX, train_y, test_y = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

    # Ensure directory exists and save the .npz file
    np.savez_compressed(save_path_train, X=trainX, y=train_y)
    np.savez_compressed(save_path_test, X=testX, y=test_y)
    print(f"Data saved to temp/...")

In [63]:
save_past_dependence_merged_data(df_list, window_size = 10)

Data saved to temp/...


In [64]:
training_data = np.load("temp/train_compressed.npz")
test_data = np.load("temp/test_compressed.npz")

trainX, train_y = training_data['X'], training_data['y']
testX, test_y = test_data['X'], test_data['y']

In [65]:
print(testX.shape)
print(trainX.shape)

(318, 10, 15)
(1272, 10, 15)


In [67]:
print(test_y.shape)
print(train_y.shape)

(318,)
(1272,)


In [68]:
testX

array([[[28.1, 29.4, 32.4, ..., 27.9, 48.1, 27.1],
        [28.1, 29.4, 32.4, ..., 27.9, 48.1, 27.1],
        [28.1, 29.4, 32.5, ..., 27.7, 47.9, 27.1],
        ...,
        [27.9, 29.4, 32.2, ..., 27.7, 47.3, 27.1],
        [27.9, 29.4, 32.2, ..., 27.7, 47.1, 27.2],
        [27.9, 29.4, 32.2, ..., 27.7, 47. , 27.1]],

       [[29. , 29.7, 34.4, ..., 29.3, 53.1, 29.4],
        [28.9, 29.7, 34.6, ..., 29.5, 53.3, 29.4],
        [28.9, 29.8, 34.7, ..., 29.5, 53.4, 29.4],
        ...,
        [28.7, 29.9, 34.9, ..., 29.5, 54. , 29.5],
        [28.8, 29.9, 34.9, ..., 29.5, 53.9, 29.4],
        [28.7, 29.9, 34.8, ..., 29.5, 54.1, 29.4]],

       [[30.1, 31.6, 36.2, ..., 30.7, 53.8, 29.9],
        [30.1, 31.6, 36.1, ..., 30.7, 54. , 29.9],
        [30. , 31.6, 36.2, ..., 30.6, 54. , 29.9],
        ...,
        [30.1, 31.7, 36.2, ..., 30.6, 54.1, 30.1],
        [30.1, 31.7, 36.4, ..., 30.8, 54. , 30.1],
        [30.1, 31.6, 36.4, ..., 30.7, 54. , 30.1]],

       ...,

       [[29.4, 31.1, 36.

In [69]:
test_y

array([24, 26, 40,  0,  7, 25, 28, 11,  8,  7, 19, 12, 24, 41, 39, 30, 10,
        8, 49, 27, 20, 48, 41, 33,  9, 43, 17, 43,  6, 18, 28, 20, 32, 12,
       19, 24, 46,  1, 18, 27,  1,  8, 13, 11, 25, 13, 47, 27,  3, 16, 25,
       43, 11, 27,  0, 39, 17, 31, 21, 15, 20, 17,  8,  1, 50, 22, 38, 12,
       18, 24, 18, 13, 14, 19, 40, 35, 12, 26, 23, 33, 42, 21, 25, 21, 24,
       34, 41, 28, 31, 37, 35, 20, 15, 48, 40, 19, 43, 19, 41, 11,  9, 39,
        4, 27, 23, 18, 28, 47, 15, 23,  7, 29, 27, 34, 24, 35, 19, 17, 27,
        7, 14, 21, 16,  1, 22, 29,  9, 11,  4, 41, 18, 33, 17, 26, 12, 10,
       42, 23, 33, 19,  9, 33, 24,  2, 19, 36, 38, 26, 36, 42, 52, 13, 14,
       46, 23, 21, 28, 26, 10, 27, 21, 21, 16, 35, 41, 21, 43, 26, 26, 16,
       17, 50, 32, 51, 42, 36, 41, 42,  3, 13,  6,  6, 49, 25, 30, 33, 24,
       18, 30, 28, 15, 28, 36, 10, 27, 29, 29, 23, 11, 34, 22,  4, 21,  9,
       39,  7, 11, 32, 41, 13, 28, 22, 18,  3, 23,  5, 39, 11, 38, 42, 21,
       26, 27, 17, 30, 13

<h2>Build Objective function</h2>

In [70]:
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping

<h4>Test a LSTM model with manual hyperparameter</h4>

In [71]:
m,n,h = trainX.shape

model = Sequential()
model.add(Input(shape=(n,h)))
model.add(LSTM(units=64))
model.add(Dropout(rate=0.1))

model.add(Dense(units=1, activation="linear"))  # Output layer

    # Compile the model
optimizer = Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss="mse")

# Early stopping
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    trainX, train_y,
    validation_split=0.2,  # Correct argument
    epochs=50,
    batch_size=16,
    callbacks=[early_stopping],
    verbose=1
    )

    # Evaluate the model
y_pred = model.predict(testX)
mse = mean_squared_error(test_y, y_pred.reshape(-1))

Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 432.0111 - val_loss: 170.9992
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 153.6616 - val_loss: 144.5192
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 146.6583 - val_loss: 143.2465
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 143.7312 - val_loss: 145.5406
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 148.5300 - val_loss: 143.8352
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 157.3051 - val_loss: 143.5559
Epoch 7/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 141.6401 - val_loss: 143.5070
Epoch 8/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 159.6006 - val_loss: 143.6440
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step


In [72]:
y_pred.shape

(318, 1)

In [73]:
mse

152.2822265625

In [74]:
# Objective function
m,n,h = trainX.shape
def objective(trial):
    # Hyperparameter tuning
    num_units = trial.suggest_int("num_units", 16, 224, step=16)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_int("batch_size", 16, 128, step=16)
    epochs = trial.suggest_int("epochs", 10, 100, step=10)
    

    # Model definition
    model = Sequential()

    if num_layers > 1:
        model.add(LSTM(units=num_units, return_sequences=True, input_shape=(n,h)))
        model.add(Dropout(rate=dropout_rate))
    else:
        model.add(LSTM(units=num_units, return_sequences=False, input_shape=(n,h)))
    
    model.add(Dropout(rate=dropout_rate))
    
    for _ in range(num_layers-2):
        model.add(LSTM(units=num_units, return_sequences=True))
        model.add(Dropout(rate=dropout_rate))

    if(num_layers > 1):
        model.add(LSTM(units=num_units, return_sequences=False))
        model.add(Dropout(rate=dropout_rate))
    model.add(Dense(units=1, activation='linear'))  # Output layer

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="mse")

    
    early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
    
    model.fit(
        trainX, train_y,
        validation_split = 0.2,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Evaluate the model
    y_pred = model.predict(testX)
    mse = mean_squared_error(test_y, y_pred)
    
    return mse

In [75]:
study = optuna.create_study(direction="minimize", study_name="Tree Parzen Optimization of LSTM")
study.optimize(objective, n_trials=50)

[I 2025-01-13 11:22:49,689] A new study created in memory with name: Tree Parzen Optimization of LSTM
/home/imran/github space/OptimizationAlgo/ConferenceEnv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step


[I 2025-01-13 11:23:13,710] Trial 0 finished with value: 152.34910583496094 and parameters: {'num_units': 192, 'num_layers': 2, 'dropout_rate': 0.19418846214335295, 'learning_rate': 0.008262160737995673, 'batch_size': 80, 'epochs': 100}. Best is trial 0 with value: 152.34910583496094.
/home/imran/github space/OptimizationAlgo/ConferenceEnv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step


[I 2025-01-13 11:23:31,711] Trial 1 finished with value: 152.30018615722656 and parameters: {'num_units': 48, 'num_layers': 3, 'dropout_rate': 0.1509907724983689, 'learning_rate': 0.003943473368871887, 'batch_size': 48, 'epochs': 60}. Best is trial 1 with value: 152.30018615722656.
/home/imran/github space/OptimizationAlgo/ConferenceEnv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

[I 2025-01-13 13:36:50,412] Trial 47 finished with value: 17.375186920166016 and parameters: {'num_units': 192, 'num_layers': 2, 'dropout_rate': 0.13042879281692688, 'learning_rate': 0.0004731185710096884, 'batch_size': 64, 'epochs': 40}. Best is trial 15 with value: 8.538989067077637.
/home/imran/github space/OptimizationAlgo/ConferenceEnv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 109ms/step


[I 2025-01-13 13:39:35,943] Trial 48 finished with value: 167.71945190429688 and parameters: {'num_units': 96, 'num_layers': 2, 'dropout_rate': 0.16037399293160362, 'learning_rate': 0.0001005012936602318, 'batch_size': 80, 'epochs': 80}. Best is trial 15 with value: 8.538989067077637.
/home/imran/github space/OptimizationAlgo/ConferenceEnv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step


[I 2025-01-13 13:42:19,009] Trial 49 finished with value: 19.84587287902832 and parameters: {'num_units': 128, 'num_layers': 2, 'dropout_rate': 0.10049839813058088, 'learning_rate': 0.00019295557853659032, 'batch_size': 48, 'epochs': 70}. Best is trial 15 with value: 8.538989067077637.


In [77]:
study.best_params

{'num_units': 96,
 'num_layers': 2,
 'dropout_rate': 0.14310311885241903,
 'learning_rate': 0.0002352362187582648,
 'batch_size': 16,
 'epochs': 90}

In [80]:
!pip install plotly

In [81]:
from optuna.visualization import plot_parallel_coordinate

fig = plot_parallel_coordinate(study)
fig.show()

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [82]:
study._get_trials

<bound method Study._get_trials of <optuna.study.study.Study object at 0x71e469131120>>

In [86]:
from optimizers import GWO

def objective_for_gwo(params):
    num_units = int(params[0])
    num_layers = int(params[1])
    dropout_rate = params[2]
    learning_rate = params[3]
    batch_size = int(params[4])
    epochs = int(params[5])
    
    # Model definition
    model = Sequential()
    model.add(Input(shape=(n, h)))

    if num_layers > 1:
        model.add(LSTM(units=num_units, return_sequences=True))
        model.add(Dropout(rate=dropout_rate))
    else:
        model.add(LSTM(units=num_units, return_sequences=False))
    model.add(Dropout(rate=dropout_rate))

    for _ in range(num_layers - 2):
        model.add(LSTM(units=num_units, return_sequences=True))
        model.add(Dropout(rate=dropout_rate))

    if num_layers > 1:
        model.add(LSTM(units=num_units, return_sequences=False))
        model.add(Dropout(rate=dropout_rate))

    model.add(Dense(units=1, activation="linear"))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="mse")

    # Train the model
    early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
    model.fit(
        trainX, train_y,
        validation_split=0.2,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping],
        verbose=0
    )

    # Predict and calculate MSE
    y_pred = model.predict(testX)
    mse = mean_squared_error(test_y, y_pred)
    return mse

In [ ]:
""""
num_units = int(params[0])
num_layers = int(params[1])
dropout_rate = params[2]
learning_rate = params[3]
batch_size = int(params[4])
epochs = int(params[5])
"""
lb = [16, 1, 0.1, 1e-4, 16, 10]  # Lower bounds
ub = [224, 3, 0.5, 1e-2, 128, 100]  # Upper bounds
dim = len(lb)

# Run GWO
best_params, best_score = GWO(objective_for_gwo, lb, ub, dim, SearchAgents_no=7, Max_iter=50)


[[5.02184362e+01 1.71492447e+00 4.60460085e-01 3.19808566e-03
  5.82552898e+01 8.17602773e+01]
 [1.04881955e+02 2.79914893e+00 1.81840555e-01 3.43337980e-03
  7.00349115e+01 7.79977892e+01]
 [2.05992971e+02 1.56239589e+00 2.31135415e-01 4.56508325e-03
  1.19780966e+02 2.88180823e+01]
 [6.57016204e+01 1.03275692e+00 3.68073342e-01 3.71944651e-04
  1.05463435e+02 6.25085057e+01]
 [1.23177515e+02 2.36822689e+00 1.10998809e-01 5.67640152e-03
  9.60258808e+01 8.23549215e+01]
 [1.52911238e+02 1.64657761e+00 3.80667711e-01 7.72510673e-03
  6.41183505e+01 8.43913139e+01]
 [2.20419418e+02 1.12805550e+00 4.34901444e-01 2.29268601e-03
  4.38274648e+01 8.83400534e+01]]
GWO is optimizing  "objective_for_gwo"
[5.02184362e+01 1.71492447e+00 4.60460085e-01 3.19808566e-03
 5.82552898e+01 8.17602773e+01]
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
[1.04881955e+02 2.79914893e+00 1.81840555e-01 3.43337980e-03
 7.00349115e+01 7.79977892e+01]
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step
[2.05992971e+02 1.56239589e+0